## Decision Trees
*Problem* : **Titanic Survior Prediction** Kaggle Challenge

### Learning Goals
- How to pre-process data? 
    - Dropping not useful features
    - Filling the missing values (Data Imputation)
    
- Creating a Binary Decision Tree from Scratch


In [1]:
import numpy as np
import pandas as pd

In [15]:
data = pd.read_csv("titanic.csv")

In [16]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
data.shape

(891, 12)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [19]:
colums_to_drop = ["PassengerId", "Name", "Ticket", "Cabin", "Embarked"]

In [20]:
data_clean = data.drop(columns=colums_to_drop)

In [24]:
data_clean.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [26]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data_clean['Sex'] =  le.fit_transform(data_clean['Sex'])

In [27]:
data_clean.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [28]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int64
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


In [31]:
data_clean= data_clean.fillna(value=data_clean['Age'].mean())

In [32]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


In [33]:
## Imputer Class

In [41]:
data_clean.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [49]:
data_clean.loc[1]

Survived     1.0000
Pclass       1.0000
Sex          0.0000
Age         38.0000
SibSp        1.0000
Parch        0.0000
Fare        71.2833
Name: 1, dtype: float64

In [50]:
data_clean.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [51]:
input_cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
output_cols = ['Survived']

In [54]:
X = data_clean[input_cols]
y = data_clean[output_cols]

## Entropy

In [62]:
def entropy(col):
    counts = np.unique(col , return_counts=True)
    N = float(col.shape[0])
    
    ent = 0.0
    
    for ix in counts[1]:
        p = ix/N
        ent += (p*np.log2(p))
        
    return -ent

In [69]:
def divide_data(x_data, fkey, fval):
    
    x_right = pd.DataFrame([], columns=x_data.columns)
    x_left = pd.DataFrame([], columns=x_data.columns)
    
    for ix in range(x_data.shape[0]):
        val = x_data[fkey].iloc[ix]
        
        if val>=fval:
            x_right = x_right.append(x_data.iloc[ix])
        else:
            x_left = x_left.append(x_data.iloc[ix])
            
    return x_left, x_right

In [71]:
# divide_data(data_clean[:10],'Sex', data_clean['Sex'].mean() )

In [73]:
def information_gain(x_data, fkey, fval):
    left, right = divide_data(x_data, fkey, fval)

    # % of total samples on left and right
    l = float(left.shape[0]/x_data.shape[0])
    r = float(right.shape[0]/x_data.shape[0])
    
    
    igain = entropy(x_data['Survived'])  - (l*entropy(left['Survived'])+ r*entropy(right['Survived']))
    return igain

In [79]:
for col in X.columns:
    ig = information_gain(data_clean, col, data_clean[col].mean())
    print(col , ig)

Pclass 0.07579362743608165
Sex 0.2176601066606142
Age 0.001158644038169343
SibSp 0.009584541813400071
Parch 0.015380754493137694
Fare 0.042140692838995464


# DecisionTree

In [125]:
class DecisionTree():
    
    def __init__(self, depth = 0, max_depth = 5):
        self.left = None
        self.right = None
        self.fkey = None
        self.fval = None
        self.depth = depth
        self.max_depth = max_depth
        self.target = None
        
    
    def train(self, X_train):
        features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
        info_gains = []
        for f in features:
            ig = information_gain(X_train, f, X_train[f].mean())
            info_gains.append(ig)
    
        self.fkey =  features[np.argmax(info_gains)]
        self.fval = X_train[self.fkey].mean()
        print("Making tree with feature", self.fkey, "depth -" , self.depth)
        
        data_left, data_right = divide_data(X_train, self.fkey, self.fval)

        data_left = data_left.reset_index(drop=True)
        data_right = data_right.reset_index(drop =True)
        
        # Recursion
        
        # Leaf Node
        if data_left.shape[0] == 0 or data_right.shape[0]==0:
            
            if X_train['Survived'].mean() >= 0.5 :
                self.target = "Survied"
            else:
                self.target = "Dead"

            return
        
        # Early stopping
        if (self.depth>=self.max_depth):
            if X_train['Survived'].mean() >= 0.5 :
                self.target = "Survied"
            else:
                self.target = "Dead"

            return
        
        # Recursions
        self.left = DecisionTree(depth=self.depth+1, max_depth=self.max_depth)
        self.left.train(data_left)
        
        self.right = DecisionTree(depth=self.depth+1, max_depth=self.max_depth)
        self.right.train(data_right)
        
        
         # Having Target for all nodes
        if X_train['Survived'].mean() >= 0.5 :
            self.target = "Survied"
        else:
            self.target = "Dead"

        return
        
        
    def predict(self, test):
        if test[self.fkey] > self.fval:
            # right side
            if self.right is None:
                return self.target
            return self.right.predict(test)

        else:
            # left side
            if self.left is None:
                return self.target
            return self.left.predict(test)

## Split

In [127]:
split = int(0.7*X.shape[0])

In [128]:
train_data  =data_clean[:split]
test_data = data_clean[split:]

In [129]:
train_data.shape

(623, 7)

In [130]:
test_data.shape

(268, 7)

In [131]:
dt = DecisionTree()

In [132]:
dt.train(train_data)

Making tree with feature Sex depth - 0
Making tree with feature Pclass depth - 1
Making tree with feature Age depth - 2
Making tree with feature SibSp depth - 3
Making tree with feature Pclass depth - 4
Making tree with feature Pclass depth - 5
Making tree with feature Age depth - 5
Making tree with feature SibSp depth - 4
Making tree with feature Parch depth - 5
Making tree with feature Pclass depth - 5
Making tree with feature SibSp depth - 3
Making tree with feature Fare depth - 4
Making tree with feature Parch depth - 5
Making tree with feature Pclass depth - 5
Making tree with feature Pclass depth - 4
Making tree with feature Pclass depth - 5
Making tree with feature Pclass depth - 5
Making tree with feature Parch depth - 2
Making tree with feature SibSp depth - 3
Making tree with feature Fare depth - 4
Making tree with feature Age depth - 5
Making tree with feature Age depth - 5
Making tree with feature Fare depth - 4
Making tree with feature Age depth - 5
Making tree with featur

In [133]:
dt.fkey

'Sex'

In [134]:
dt.left.fkey

'Pclass'

In [135]:
dt.right.fkey

'Fare'

## Predict

In [137]:
y_pred = []

for ix in range(test_data.shape[0]):
    p = dt.predict(test_data.iloc[ix])
    y_pred.append(p)

In [138]:
y_pred

['Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survied',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survied',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survied',
 'Dead',
 'Dead',
 'Survied',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survied',
 'Dead',
 'Survied',
 'Dead',
 'Survied',
 'Survied',
 'Dead',
 'Dead',
 'Survied',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survied',
 'Survied',
 'Survied',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survied',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survied',
 'Survied',
 'Survied',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survied',
 'Dead',
 'Dead',
 'Survied',
 'Dead',
 'Survied',
 'Dead',
 'Dead',
 'Dead',
 'Survied',
 'Dead',
 'Survied',
 'Dead',
 'Survied',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survied',
 'Survied',
 'Dead',
 'Dead',
 'Survied',
 'Dead',
 'Dead',
 'Dead',
 'Survied',
 'De

In [148]:
y_pred = le.fit_transform(y_pred)

In [146]:
y_test = test_data['Survived'].values

In [151]:
(y_pred== y_test).sum()/y_test.shape[0]

0.8171641791044776

# From sklearn

In [152]:
from sklearn.tree import DecisionTreeClassifier

In [164]:
dtc = DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [165]:
dtc.fit(train_data[input_cols], train_data[output_cols])

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [166]:
y_p = dtc.predict(test_data[input_cols])

In [167]:
(y_p== y_test).sum()/y_test.shape[0]

0.8283582089552238

In [168]:
dtc.score(train_data[input_cols], train_data[output_cols])

0.8443017656500803

In [169]:
dtc.score(test_data[input_cols], test_data[output_cols])

0.8283582089552238